In [1]:
# 📌 [기본 패키지 임포트]
import os
import shutil
import torch
import ultralytics
from ultralytics import YOLO
import time
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from glob import glob

In [2]:
# 📌 [1] GPU 및 환경 체크
print("✅ CUDA 사용 가능:", torch.cuda.is_available())
print("✅ PyTorch 버전:", torch.__version__)
ultralytics.checks()

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
Setup complete  (24 CPUs, 31.7 GB RAM, 24.4/735.1 GB disk)


- Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
- Setup complete  (24 CPUs, 31.7 GB RAM, 196.5/217.8 GB disk)

In [3]:
# 📌 [2] 실패한 이전 runs 삭제
def clean_incomplete_runs(base_dir='../code/runs/detect'):
    if not os.path.exists(base_dir):
        print(f"📂 '{base_dir}' 폴더가 없어 삭제 생략")
        return

    deleted = False
    for folder in os.listdir(base_dir):
        path = os.path.join(base_dir, folder)
        best_model = os.path.join(path, 'weights', 'best.pt')
        if not os.path.exists(best_model):
            print(f"🧹 삭제 대상: {path}")
            shutil.rmtree(path)
            deleted = True

    if not deleted:
        print("✅ 삭제할 미완성 run 없음. 모든 run이 best.pt 포함.")

clean_incomplete_runs()

📂 '../code/runs/detect' 폴더가 없어 삭제 생략


In [4]:
# 📌 [3] 가장 최근 run 결과 요약 출력
def get_last_run_csv():
    runs = sorted(glob('runs/detect/*/results.csv'), key=os.path.getmtime, reverse=True)
    if runs:
        print(f"📄 마지막 훈련 결과 파일: {runs[0]}")
        df = pd.read_csv(runs[0])
        print("🔁 마지막 훈련 결과 (마지막 5줄):")
        print(df.tail())
    else:
        print("ℹ️ 이전 훈련 결과 없음.")

get_last_run_csv()

ℹ️ 이전 훈련 결과 없음.


In [5]:
# 📌 [4] 자동 넘버링 run 이름 생성
def get_next_run_name(base_dir='runs/detect', prefix='yolov11n_custom'):
    i = 1
    while os.path.exists(os.path.join(base_dir, f'{prefix}_{i:03d}')):
        i += 1
    return f'{prefix}_{i:03d}'

run_name = get_next_run_name()
print(f"🚀 새 훈련 run 이름: {run_name}")

🚀 새 훈련 run 이름: yolov11n_custom_001


In [6]:
# 📌 [5] 입력 데이터 정보 출력
def get_dataset_info(img_dir, label_dir):
    img_files = glob(os.path.join(img_dir, '*.jpg'))
    label_files = glob(os.path.join(label_dir, '*.txt'))

    total_img_size = sum(os.path.getsize(f) for f in img_files)
    total_label_size = sum(os.path.getsize(f) for f in label_files)

    print(f"\n📁 이미지 경로: {img_dir}")
    print(f"🖼️ 이미지 개수: {len(img_files)}")
    print(f"💾 이미지 용량: {total_img_size / 1024 / 1024:.2f} MB")
    print(f"📄 라벨 경로: {label_dir}")
    print(f"📝 라벨 개수: {len(label_files)}")
    print(f"💾 라벨 용량: {total_label_size / 1024:.2f} KB")

get_dataset_info('../train/images', '../train/labels')
get_dataset_info('../valid/images', '../valid/labels')


📁 이미지 경로: ../train/images
🖼️ 이미지 개수: 3530
💾 이미지 용량: 67.55 MB
📄 라벨 경로: ../train/labels
📝 라벨 개수: 3530
💾 라벨 용량: 319.50 KB

📁 이미지 경로: ../valid/images
🖼️ 이미지 개수: 801
💾 이미지 용량: 15.38 MB
📄 라벨 경로: ../valid/labels
📝 라벨 개수: 801
💾 라벨 용량: 69.86 KB


train
- 📁 이미지 경로: ../train/images
- 🖼️ 이미지 개수: 3530
- 💾 이미지 용량: 67.55 MB
- 📄 라벨 경로: ../train/labels
- 📝 라벨 개수: 3530
- 💾 라벨 용량: 319.50 KB


valid
- 📁 이미지 경로: ../valid/images
- 🖼️ 이미지 개수: 801
- 💾 이미지 용량: 15.38 MB
- 📄 라벨 경로: ../valid/labels
- 📝 라벨 개수: 801
- 💾 라벨 용량: 69.86 KB

## YOLO 모델 로드 및 학습

In [7]:
import os

def safe_delete_cache(path: str):
    """
    조건에 따라 .cache 파일을 삭제:
    - 파일이 존재하고
    - 용량이 1KB 이하이거나
    - 생성된 지 1시간 이상 되었거나
    - 'force_delete_cache=True'로 설정 시
    삭제를 수행함.
    """
    if not os.path.exists(path):
        print(f"🟢 캐시 파일 없음 (삭제 불필요): {path}")
        return

    file_size = os.path.getsize(path)
    last_modified = os.path.getmtime(path)
    now = os.path.getmtime(__file__) if '__file__' in globals() else None

    # 사용자 조건: 파일이 너무 작거나 오래되었을 때만 삭제
    should_delete = (
        file_size < 1024 or  # 너무 작으면 손상 가능성
        (now and (now - last_modified > 3600))  # 1시간 이상 경과 시
    )

    if should_delete:
        try:
            os.remove(path)
            print(f"✅ 캐시 삭제 완료: {path}")
        except Exception as e:
            print(f"❌ 캐시 삭제 실패: {path} → {e}")
    else:
        print(f"🔒 캐시 유지: {path} (조건 미충족)")

# 삭제 시도
safe_delete_cache(r"D:\Project\PJT_07\train\labels.cache")
safe_delete_cache(r"D:\Project\PJT_07\valid\labels.cache")

🔒 캐시 유지: D:\Project\PJT_07\train\labels.cache (조건 미충족)
🔒 캐시 유지: D:\Project\PJT_07\valid\labels.cache (조건 미충족)


In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
import torch
import time
import threading
from ultralytics import YOLO

# GPU 메모리 사용량 주기적으로 출력하는 함수
def gpu_monitor():
    while True:
        if torch.cuda.is_available():
            mem_alloc = torch.cuda.memory_allocated() / 1024**2
            mem_reserved = torch.cuda.memory_reserved() / 1024**2
            print(f"[GPU 모니터] 메모리 할당: {mem_alloc:.1f}MB, 예약: {mem_reserved:.1f}MB")
        time.sleep(10)  # 10초마다 출력

# GPU 모니터 스레드 시작 (데몬 스레드라 메인 종료 시 같이 종료됨)
monitor_thread = threading.Thread(target=gpu_monitor, daemon=True)
monitor_thread.start()

# 모델 로드
model = YOLO('yolo11n.pt')
# 이전 학습 모델에서 시작하려면 아래처럼 경로 지정
# model = YOLO('runs/detect/yolov11n_custom_001/weights/best.pt')

run_name = "yolov11n_custom_001"  # 예시로 이름 지정

start_time = time.time()

model.train(
    data='data.yaml',
    epochs=30,
    imgsz=640,
    batch=8,
    name=run_name,
    device='0',
    workers=0,
    verbose=True  # 더 자세한 로그 출력
)

train_duration = time.time() - start_time
print(f"✅ 훈련 소요 시간: {train_duration:.2f}초")

[GPU 모니터] 메모리 할당: 0.0MB, 예약: 0.0MB
Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11n_custom_001, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

train: Scanning D:\Project\PJT_07\train\labels... 3530 images, 3 backgrounds, 0 corrupt: 100%|██████████| 3530/3530 


train: New cache created: D:\Project\PJT_07\train\labels.cache
val: Fast image access  (ping: 0.20.0 ms, read: 47.926.6 MB/s, size: 20.7 KB)


val: Scanning D:\Project\PJT_07\valid\labels...:   0%|          | 0/801 [00:00<?, ?it/s]

[GPU 모니터] 메모리 할당: 52.1MB, 예약: 98.0MB


...

[GPU 모니터] 메모리 할당: 713.5MB, 예약: 1902.0MB


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  65%|██████▍   | 33/51 [00

[GPU 모니터] 메모리 할당: 684.2MB, 예약: 1902.0MB


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00


[GPU 모니터] 메모리 할당: 645.4MB, 예약: 1902.0MB
                   all        801        944      0.955      0.873      0.946      0.816
           Green Light         87        122      0.826      0.739      0.826      0.473
             Red Light         74        108      0.901      0.713      0.831      0.506
       Speed Limit 100         52         52      0.924      0.923      0.973       0.87
       Speed Limit 110         17         17          1      0.882      0.962       0.89
       Speed Limit 120         60         60      0.982        0.9      0.976      0.891
        Speed Limit 20         56         56      0.982      0.893      0.979      0.859
        Speed Limit 30         71         74      0.951      0.959      0.982      0.914
        Speed Limit 40         53         55      0.981      0.926      0.983      0.867
        Speed Limit 50         68         71      0.924      0.853      0.953      0.844
        Speed Limit 60         76         76      0.969      0.908    

## 검증 (val)

In [12]:
# 📌 [7] 훈련된 모델에 대해 별도 검증 수행
start_val = time.time()

# 가장 성능이 좋았던 가중치 불러오기
model = YOLO(f'runs/detect/{run_name}/weights/best.pt')

# 검증 수행 (data.yaml에서 val 경로 사용)
results = model.val()
val_duration = time.time() - start_val
print(f"✅ 검증 소요 시간: {val_duration:.2f}초")

# 선택 출력: 주요 지표
print(f"mAP50: {results.box.map50:.4f}, mAP50-95: {results.box.map:.4f}")

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,585,077 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 68.633.3 MB/s, size: 18.1 KB)


val: Scanning D:\Project\PJT_07\valid\labels.cache... 801 images, 0 backgrounds, 0 corrupt: 100%|██████████| 801/801
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00


                   all        801        944       0.95      0.874      0.945      0.817
           Green Light         87        122      0.834      0.746      0.822      0.474
             Red Light         74        108      0.896      0.714      0.832      0.509
       Speed Limit 100         52         52      0.923      0.923      0.974       0.87
       Speed Limit 110         17         17      0.938       0.89      0.962       0.89
       Speed Limit 120         60         60      0.979        0.9      0.976      0.894
        Speed Limit 20         56         56      0.981      0.893      0.979       0.86
        Speed Limit 30         71         74       0.95      0.959      0.982      0.914
        Speed Limit 40         53         55       0.98      0.927      0.983      0.869
        Speed Limit 50         68         71      0.924      0.857      0.953      0.844
        Speed Limit 60         76         76      0.966      0.908      0.949      0.869
        Speed Limit 7

## 테스트 이미지 예측

In [16]:
# 📌 [8] 예측 실행
test_img = '../test/images/00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.jpg'

# ✅ 예측 이미지 저장
pred = model.predict(
    source=test_img,
    conf=0.25,
    save=True,                            # 예측 이미지 저장 활성화
    project='runs/predict',               # 저장할 상위 폴더 (디폴트: 'runs/predict')
    name='custom_test',                   # 하위 폴더명 (ex. runs/predict/custom_test)
    exist_ok=True                         # 이미 존재하는 폴더에 덮어쓰기 허용
)


image 1/1 D:\Project\PJT_07\code\..\test\images\00014_00004_00020_png.rf.e3b35f3ca153fde99af4db4a67d2a564.jpg: 640x640 1 Stop, 28.1ms
Speed: 9.1ms preprocess, 28.1ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\predict\custom_test


In [17]:
# 📌 [9] 예측 결과 시각화
for r in pred:
    print("🎯 예측된 클래스 목록:", r.names)
    r.show()

🎯 예측된 클래스 목록: {0: 'Green Light', 1: 'Red Light', 2: 'Speed Limit 10', 3: 'Speed Limit 100', 4: 'Speed Limit 110', 5: 'Speed Limit 120', 6: 'Speed Limit 20', 7: 'Speed Limit 30', 8: 'Speed Limit 40', 9: 'Speed Limit 50', 10: 'Speed Limit 60', 11: 'Speed Limit 70', 12: 'Speed Limit 80', 13: 'Speed Limit 90', 14: 'Stop'}


## ONNX 모델로 내보내기

In [18]:
# 📌 [10] ONNX 모델로 내보내기
export_path = model.export(format='onnx')
print(f"🧠 ONNX 모델 저장 경로: {export_path}")

Ultralytics 8.3.156  Python-3.11.9 torch-2.7.1+cu128 CPU (13th Gen Intel Core(TM) i7-13700HX)

PyTorch: starting from 'runs\detect\yolov11n_custom_001\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 19, 8400) (5.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.56', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 41.5 MB/s eta 0:00:00
   ---------------------------------------- 214.9/214.9 MB 38.6 MB/s eta 0:00:00
   ---------------------------------------- 6/6 [onnxruntime-gpu]

requirements: AutoUpdate success  42.1s
WARNING requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success  47.0s, saved as 'runs\detect\yolov11n_custom_001\weights\best.onnx' (10.1 MB)

Export complete (48.0s)
Results saved to D:\Project\PJT_07\code\runs\

## ONNX 모델을 통한 시각화 예측 (cv2 기반)

In [19]:
# 📌 [11] ONNX 모델 기반 예측 시각화
onnx_model = YOLO(os.path.join('runs/detect', run_name, 'weights', 'best.onnx'))

image = cv2.imread(test_img)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
resized_image = cv2.resize(image, (640, 640))

results = onnx_model(resized_image, imgsz=640)

for result in results:
    boxes = result.boxes
    names = result.names
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        cls = int(box.cls[0])
        label = f"{names[cls]} {conf:.2f}"

        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(image, (x1, y1 - text_height - 5), (x1 + text_width, y1), (0, 255, 0), -1)
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis('off')
plt.title("🔍 ONNX 예측 결과")
plt.show()

Loading runs\detect\yolov11n_custom_001\weights\best.onnx for ONNX Runtime inference...
Using ONNX Runtime CUDAExecutionProvider

0: 640x640 1 Stop, 33.0ms
Speed: 8.8ms preprocess, 33.0ms inference, 5.2ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 1200x1200 with 1 Axes>

In [21]:
# import pandas as pd

# results_csv = f'runs/detect/{run_name}/results.csv'
# df = pd.read_csv(results_csv)
# print("📋 CSV 컬럼 목록:")
# print(df.columns)

📋 CSV 컬럼 목록:
Index(['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss',
       'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)',
       'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss',
       'lr/pg0', 'lr/pg1', 'lr/pg2'],
      dtype='object')


In [22]:
# 📌 [12] 훈련 결과 시각화 (mAP/Loss 등)
results_csv = f'runs/detect/{run_name}/results.csv'

if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    print(f"📈 훈련 결과 요약 (Epoch별 mAP):\n{df[['metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail()}")

    # Box Loss 그래프
    df[['train/box_loss', 'val/box_loss']].plot(title='Box Loss', figsize=(6, 4))
    plt.savefig(f'runs/detect/{run_name}/box_loss.png')
    plt.close()

    # mAP 그래프
    df[['metrics/mAP50(B)', 'metrics/mAP50-95(B)']].plot(title='mAP', figsize=(6, 4))
    plt.savefig(f'runs/detect/{run_name}/map.png')
    plt.close()

else:
    print("📛 results.csv 없음 (훈련 실패?)")

📈 훈련 결과 요약 (Epoch별 mAP):
    metrics/mAP50(B)  metrics/mAP50-95(B)
25           0.93757              0.80491
26           0.93899              0.80909
27           0.94335              0.81435
28           0.94540              0.81500
29           0.94552              0.81616


In [23]:
best_idx = df['metrics/mAP50-95(B)'].idxmax()
print("🎯 최고 성능 Epoch:")
print(df.loc[best_idx, ['epoch', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']])

🎯 최고 성능 Epoch:
epoch                  30.00000
metrics/mAP50(B)        0.94552
metrics/mAP50-95(B)     0.81616
Name: 29, dtype: float64


In [24]:
# 📌 [13] 결과를 JSON으로 저장
def save_results_to_json(csv_path, json_path):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        result_data = df.to_dict(orient='records')
        with open(json_path, 'w') as f:
            json.dump(result_data, f, indent=4)
        print(f"📦 결과 JSON 저장 완료: {json_path}")

save_results_to_json(
    csv_path=f'runs/detect/{run_name}/results.csv',
    json_path=f'runs/detect/{run_name}/results_summary.json'
)

📦 결과 JSON 저장 완료: runs/detect/yolov11n_custom_001/results_summary.json
